<a href="https://colab.research.google.com/github/amyth18/deep-learning/blob/main/CS598_Deep_Learning_For_Healthcare_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Load Data

In [ ]:
# read data
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/dataset/NOTEEVENTS.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.head(2)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...


# Data Preprocessing
Need to focus on 2 tables
1. NOTESEVENTS.csv
2. DIAGNOSES_ICD.csv

Join tables by <subject_id, hadm_id>

Construct 2 datasets from "TEXT" field in NOTESEVENTS.csv for each <subject_id, hadm_id> pair (i.e discharge summary for that admission.)

X1, y and X2, y
x1 = sequence of vectors from word2vec 
x2 = sequence of vectors from tf-idf
y = list of icd codes for <subject_id, hadm_id> i.e. diagnosis maded in ICU admission.

Need to focus on 50 and 100 most commonly diagnosed diseases.

## Common data preprocessing
1. Use NLTK + MetaMap to extract only the symptom related entities (how to use MetaMap is unknown atm.)

2. Filter out sections in discharge summaries that are related to symptoms only, ignore others to speed up things.

3. Negative filters (e.g. "no sign of breath problem").

## Data processing for X1 (Word2Vec)
1. Generate Word2Vec embeddings (currently using Gensim) using "TEXT".

## Data processing for X2 (TF-IDF)
1. Generate TF-IDF vector for each symptom entity.

## Data processing for y
1. multi-hot encoding?
 




In [4]:
import re
import nltk

from nltk.corpus import stopwords
nltk.download('stopwords')

eng_stop_words =  stopwords.words('english')

class MySentences(object):
    def __init__(self, dframe):
        self.dframe = dframe
    
    # TODO: Keeping only alpha numeric characters and spaces for now.
    # need to make this better. Find some good libraries.
    def sanitize_text(self, text):
      test = text.strip()
      text = re.sub(r'\s\s+', ' ', text)
      text = re.sub(r'[^a-zA-z0-9\/\.\?\!\s;,\'\-]', '', text)
      text = re.sub(r'[\.\-\/\?\!;,]', ' ', text)
      text = re.sub(r'[\[\]]', '', text)
      return text

    # TODO: adding some basic checks again need to make it better.
    def sanitize_words(self, sentence):
      return [w for w in sentence if w not in eng_stop_words and not w.isdigit()]

    def __iter__(self):
        for idx in range(len(self.dframe)):
          text = self.sanitize_text(self.dframe["TEXT"][idx])

          yield self.sanitize_words(text.split())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Test code for sentence generator

In [ ]:
sgen = MySentences(df)
sentences = list()
for idx, s in enumerate(sgen):
  print(s)
  if idx == 1:
    break

Word2Vec training using gensim.

In [5]:
import gensim

sgen = MySentences(df) # a memory-friendly iterator
model = gensim.models.Word2Vec(sgen, min_count=1, workers=4)
model.save("word2vec.model")

KeyboardInterrupt: ignored

In [ ]:
!ls /content/drive/MyDrive/dataset

mimic.zip  NOTEEVENTS.csv
